In [13]:
from langchain.agents import create_agent
from langchain.tools import tool
from dotenv import load_dotenv
from typing import Dict, Any
from tavily import TavilyClient
from langgraph.checkpoint.memory import InMemorySaver
from pydantic import BaseModel

MODELO = "gpt-5-nano"

SYSTEM_PROMPT = """
Você é um chefe de cozinha experiente que ajuda as pessoas a encontrar receitas deliciosas e acessíveis com base nos ingredientes que as pessoas possuem. 
Você deve fornecer receitas detalhadas, dicas de preparo e sugestões de substituições de ingredientes quando necessário.
Além disso, você deve ser capaz de pesquisar na web para encontrar receitas adicionais e informações relevantes para ajudar os usuários a cozinhar melhor.
"""

load_dotenv()
tavily_client = TavilyClient()
config = {"configurable": {"thread_id": "1"}}

class receita(BaseModel):
    titulo: str
    ingredientes: Dict[str, str]
    instrucoes: str
    source_lik: str

@tool
def search_web(query: str) -> Dict[str, Any]:
    """
    Search the web for the user's query.
    
    Args:
        query: Search terms to look for
    """
    
    return tavily_client.search(query=query)

agent = create_agent(
    model=MODELO,
    tools=[search_web],
    checkpointer=InMemorySaver(),
    system_prompt=SYSTEM_PROMPT,
    response_format=receita   
)

In [14]:
text_content = "Analise a imagem e liste os ingredientes visíveis, e em seguida busque na internet receitas reais que utilizem esses ingredientes"

message = {
    "role": "user",
    "content": [
        {"type": "text", "text": text_content},
        {"type": "image", "url": "https://i0.wp.com/www.morganizewithme.com/wp-content/uploads/2017/09/Pantry1.jpg?ssl=1"},
    ]
}

response = agent.invoke(
    {'messages':[message]},
    config=config
)

print(response['messages'][-1].content)

{"titulo":"Sopa de Batata com Maçã","ingredientes":{"batatas":"600 g, descascadas e cortadas em cubos","maçãs":"2 médias, descascadas e cortadas (sem sementes)","cebola":"1 média, picada","alho":"2 dentes, picados","azeite de oliva":"2 colheres de sopa","caldo de legumes":"1 litro (ou água com cubos de caldo)","sal":"a gosto","pimenta-do-reino":"a gosto","creme de leite (opcional)":"para servir"},"instrucoes":"1) Aqueça o azeite em uma panela grande; refogue a cebola e o alho até ficarem translúcidos. 2) Adicione as batatas e as maçãs; refogue por 2-3 minutos. 3) Despeje o caldo e cozinhe em fogo médio até as batatas ficarem macias (cerca de 15-20 minutos). 4) Bata a sopa com um mixer de mão ou liquidificador até ficar cremosa. 5) Tempere com sal e pimenta; se desejar, incorpore creme de leite para mais brilho. 6) Sirva quente. Dicas de substituição: use abóbora ou cenoura em vez de maçã para um sabor diferente; substitua o caldo por água com cubos de caldo; ajuste a consistência adici

In [26]:
from pprint import pprint
pprint(response['structured_response'].source_lik)


'https://www.chasingtheseasons.com/potato-and-apple-soup/'


Para uma receita que é tanto deliciosa quanto econômica, recomendo o **Arroz Frito com Ovo**. É uma opção prática e barata, geralmente requerendo apenas alguns ingredientes básicos:

### Arroz Frito com Ovo

#### Ingredientes:
- 2 xícaras de arroz cozido (sobras são ótimas!)
- 2 ovos
- 1 cebola picada
- 2 dentes de alho picados
- 1/2 xícara de legumes (como cenoura, ervilha, ou o que você tiver)
- Óleo (ou manteiga) para refogar
- Sal e pimenta a gosto
- Molho de soja (opcional para temperar)

#### Modo de Preparo:
1. **Refogue** a cebola e o alho no óleo até ficarem dourados.
2. **Adicione os legumes** e refogue por alguns minutos.
3. **Empurre os legumes para o lado da panela** e quebre os ovos no espaço livre. Mexa-os até cozinhar.
4. **Misture o arroz** cozido e tempere com sal, pimenta e molho de soja.
5. **Frite tudo juntos** por alguns minutos até aquecer bem.

### Por que é uma boa escolha:
- **Econômica**: Utiliza arroz já cozido e ingredientes simples.
- **Prática**: Rápida d